In [1]:
import re

import datasets
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix,cohen_kappa_score, mean_squared_error


In [2]:
models = ['mistral', 'llama3-8b', 'mixtral-8x22b', 'llama3-70b']
splits = ['ua', 'uq']
configs = ['zeroshot', 'rag_3', 'rag_5']

In [ ]:
sacrebleu = datasets.load_metric('sacrebleu')
rouge = datasets.load_metric('rouge')
meteor = datasets.load_metric('meteor')
bert_score = datasets.load_metric('bertscore')

for split in splits:
    print(f"\n{'='*20} {split.upper()} {'='*20}\n")
    for model in models:
        print(f"\n{'-'*10} Model: {model.upper()} {'-'*10}\n")
        for config in configs:
            print(f"\n{'*'*5} Configuration: {config} {'*'*5}\n")
            filename = f"generated_results/{split}/{split}_{config}_{model}.csv"
            df = pd.read_csv(filename)

            # Scoring Metrics
            print("### Performance Metrics ###")
            print(f"Accuracy: {round(accuracy_score(y_true=df['label'], y_pred=df['clean_pred_label'], normalize=True), 3)}")
            print(f"F1 Score (macro): {round(f1_score(df['label'], df['clean_pred_label'], average='macro'), 3)}")
            print(f"Quadratic Weighted Kappa (QWK): {round(cohen_kappa_score(df['label'], df['pred_label'], weights='quadratic'), 3)}")
            # print(f"Root Mean Square Error (RMSE): {round(root_mean_squared_error(df['score'], df['pred_score'], squared=False), 3)}\n")

            rmse = np.sqrt(mean_squared_error(df['score'], df['pred_score']))
            print(f"Root Mean Square Error (RMSE): {round(rmse, 3)}\n")


            # Statistical Feedback Metrics
            references = df['feedback'].tolist()
            candidates = df['pred_feedback'].tolist()
            reference_list = [[ref] for ref in references]

            print("### Feedback Evaluation Metrics ###")
            sacrebleu_score = sacrebleu.compute(predictions=candidates, references=[[x] for x in references])['score']
            print(f"SacreBLEU Score: {round(sacrebleu_score, 3)}")

            rouge_score = rouge.compute(predictions=candidates, references=references)['rouge2']
            print(f"ROUGE-2 F1 Score: {round(rouge_score.mid.fmeasure, 3)}")

            meteor_score = meteor.compute(predictions=candidates, references=references)['meteor']
            print(f"METEOR Score: {round(meteor_score, 3)}")

            bert_f1 = np.array(bert_score.compute(predictions=candidates, references=references, lang='en', rescale_with_baseline=True)['f1']).mean().item()
            print(f"BERT Score (F1): {round(bert_f1, 3)}\n")

            print("-" * 40)

AttributeError: module 'datasets' has no attribute 'load_metric'